In [273]:
import importlib
import extraction_functions_praat
importlib.reload(extraction_functions_praat)

<module 'extraction_functions_praat' from '/home/dene/rp2/extraction_functions_praat.py'>

In [ ]:
from pathlib import Path
import pandas as pd
from extraction_functions_praat import PP_f0_mean_murton, PP_f0_sd_murton, PP_jitter, PP_f0_mean, PP_f0_sd

In [275]:
audio_dir = Path('audio_files_pre/')
features_dir = Path('extracted_features/')
segments_dir = Path('audio_files_segments/')
best_segments_dir = Path('audio_files_segments_best/')

In [277]:
AVAILABLE_FEATURES = [
    'PP_F0', 'PP_F0_SD', 
    'PP_F0_M', 'PP_F0_SD_M', 
    'PP_DUR', 
    'PP_JIT',
] ## add more

selected_features_dict = {
    'PP_F0': True, 'PP_F0_SD': True, 
    'PP_F0_M': True, 'PP_F0_SD_M': True, 
    'PP_DUR': False,
    'PP_JIT': True,
} ## enable the ones you want

In [278]:
def extract_features_vowels(audio_file, selected_features, f0_min, f0_max, jitter_type):
    extracted_features = {}
    
    if 'PP_F0' in selected_features:
        extracted_features['PP_F0'] = PP_f0_mean(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_SD' in selected_features:
        extracted_features['PP_F0_SD'] = PP_f0_sd(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_M' in selected_features:
        extracted_features['PP_F0_M'] = PP_f0_mean_murton(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_SD_M' in selected_features:
        extracted_features['PP_F0_SD_M'] = PP_f0_sd_murton(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_JIT' in selected_features:
        jitter_values = PP_jitter(audio_file, f0_min=f0_min, f0_max=f0_max, type=jitter_type)
        if jitter_type == 'all':
            extracted_features['PP_JIT_local'] = jitter_values[0]
            extracted_features['PP_JIT_abs'] = jitter_values[1]
            extracted_features['PP_JIT_rap'] = jitter_values[2]
            extracted_features['PP_JIT_ppq5'] = jitter_values[3]
            extracted_features['PP_JIT_ddp'] = jitter_values[4]
        else:
            extracted_features[f'PP_JIT_{jitter_type}'] = jitter_values
            
    return extracted_features


def extract_features_mpt(audio_file, selected_features, f0_min, f0_max, jitter_type):
    extracted_features = {}
    
    if 'PP_F0' in selected_features:
        extracted_features['PP_F0_SD'] = PP_f0_sd(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_SD_M' in selected_features:
        extracted_features['PP_F0_SD_M'] = PP_f0_sd_murton(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_JIT' in selected_features:
        jitter_values = PP_jitter(audio_file, f0_min=f0_min, f0_max=f0_max, type=jitter_type)
        if jitter_type == 'all':
            extracted_features['PP_JIT_local'] = jitter_values[0]
            extracted_features['PP_JIT_abs'] = jitter_values[1]
            extracted_features['PP_JIT_rap'] = jitter_values[2]
            extracted_features['PP_JIT_ppq5'] = jitter_values[3]
            extracted_features['PP_JIT_ddp'] = jitter_values[4]
        else:
            extracted_features[f'PP_JIT_{jitter_type}'] = jitter_values
            
    return extracted_features


def extract_features_sentences(audio_file, selected_features, f0_min, f0_max, jitter_type):
    extracted_features = {}
    
    if 'PP_F0' in selected_features:
        extracted_features['PP_F0'] = PP_f0_sd(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_JIT' in selected_features:
        jitter_values = PP_jitter(audio_file, f0_min=f0_min, f0_max=f0_max, type=jitter_type)
        if jitter_type == 'all':
            extracted_features['PP_JIT_local'] = jitter_values[0]
            extracted_features['PP_JIT_abs'] = jitter_values[1]
            extracted_features['PP_JIT_rap'] = jitter_values[2]
            extracted_features['PP_JIT_ppq5'] = jitter_values[3]
            extracted_features['PP_JIT_ddp'] = jitter_values[4]
        else:
            extracted_features[f'PP_JIT_{jitter_type}'] = jitter_values
            
    return extracted_features

In [ ]:
def process_audio_files(directory, selected_features, f0_min, f0_max, jitter_type):
    patient_dfs = {}

    files = [file for file in directory.rglob('*') if file.is_file()]

    for file in files:
        filename = file.stem.replace("_pre", "")
        parts = filename.split("_")
        if len(parts) != 4:
            print(f"Unexpected named audio file: {file}")
            continue

        patient_id, day, exercise, take_letter = parts

        file_path = str(file)
        
        if exercise == 'VOW':
            if directory == best_segments_dir:
                if file_path[-1].isdigit(): 
                    continue ## we do not analyze the best segment of the entire vowel exercise, only of the best segments
                if file_path[-1].isalpha():
                    features = extract_features_vowels(file_path, selected_features, f0_min, f0_max, jitter_type)
            else:
                features = extract_features_vowels(file_path, selected_features, f0_min, f0_max, jitter_type)
        elif exercise == 'MPT':
            features = extract_features_mpt(file_path, selected_features, f0_min, f0_max, jitter_type)
        elif exercise == 'SNT':
            features = extract_features_sentences(file_path, selected_features, f0_min, f0_max, jitter_type)
            
        df_entry = {'day': int(day), **features}    
            
        df_key = (patient_id, take_letter, exercise)
        if df_key not in patient_dfs:
            patient_dfs[df_key] = pd.DataFrame(columns=['day'] + list(features.keys()))
        
        patient_dfs[df_key] = pd.concat([patient_dfs[df_key], pd.DataFrame([df_entry])], ignore_index=True)
        
        for key, df in patient_dfs.items():
            patient_dfs[key] = df.sort_values(by='day', ascending=True).reset_index(drop=True)

        for (patient_id, take_letter, exercise), df in patient_dfs.items():
            file_name = f"{patient_id}_{exercise}_{take_letter}.csv"
            file_path = features_dir / exercise / patient_id / file_name

            df.to_csv(file_path, index=False)

In [ ]:
process_audio_files(
    directory=best_segments_dir, ## audio_dir (all pre-emphasized recordings); segments_dir (all entire vowels); best_segments_dir (all 1-sec segments with lowest jitter)
    selected_features=selected_features_dict,
    f0_min=60, f0_max=300, 
    jitter_type='abs') ## choose from 'local' 'abs' 'rap' 'ppq5' 'ddp' 'all'

/tmp/ipykernel_107442/2875657790.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  patient_dfs[df_key] = pd.concat([patient_dfs[df_key], pd.DataFrame([df_entry])], ignore_index=True)
/tmp/ipykernel_107442/2875657790.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  patient_dfs[df_key] = pd.concat([patient_dfs[df_key], pd.DataFrame([df_entry])], ignore_index=True)
/tmp/ipykernel_107442/2875657790.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is d